In [57]:
import json, os, glob
from tqdm import tqdm_notebook as tqdm
import pandas as pd
from collections import Counter
from pathlib import Path

rows = []
links = []
#num = Path('2020-03-19.json').stat().st_size
count=0

json_dir = 'en'
# Uses the json_dir variable to navigate to the dataset directory and 'get' all json files
json_pattern = os.path.join(json_dir, '*.json')

# Uses Glob with the json pattern variable to place all the json files and their directories in a list
file_list = glob.glob(json_pattern)

# directory to save data to
save_dir = "allLinks.03.2020.EnglishTweets.ftr"

# Opens the JSON file
for file in file_list:
    with open(file, 'r') as f:
        for line in f:
            if count > 10000:
                break
            count = 0
            data = json.loads(line)

            if len(data["entities"]["urls"]) > 0:
                count+=1

            if count == 1:
                #username
                user_id = data["user"]["screen_name"]
                #location
                user_location = data["user"]["location"]
                #Tweet ID
                tweet_id = data["id_str"]
                #Hashtags
                hashtags = data["entities"]["hashtags"]
                #tweet text
                urls = data["entities"]["urls"]
                rows.append((urls[0].get("expanded_url")))

pd.options.display.max_colwidth = 500
datas = pd.DataFrame(rows, columns=["urls"])
datas.to_feather(save_dir)
display(datas)

,urls
0,https://twitter.com/i/web/status/1242964476669898754
1,https://conservativechoicecampaign.com/fast-and-furious-the-deep-state-playbook-revealed/
2,https://twitter.com/i/web/status/1242964474723794944
3,https://twitter.com/i/web/status/1242964475445051398
4,https://www.politicshome.com/news/article/italian-officials-demand-health-minister-nadine-dorries-delete-fake-news-about-coronavirus-patients#.Xnvwe3DEjaI.twitter
...,...
775410,https://twitter.com/i/web/status/1245138810297606149
775411,https://twitter.com/i/web/status/1245138809290973187
775412,https://twitter.com/i/web/status/1245138810863665152
775413,https://twitter.com/i/web/status/1245138811362906112


In [1]:
# Find specific element [H1]

import re
import pandas as pd
import feather
from collections import Counter
import requests
from bs4 import BeautifulSoup


# adjustable file in ftr format
save_dir = "allLinks.03.2020.EnglishTweets.ftr"

# reads contents of save_dir.ftr
read_ftr = pd.read_feather(save_dir, columns=None, use_threads=True);

# finds 100 most common words
countList = Counter(" ".join(read_ftr["urls"]).split()).most_common(100)
f = open("english_Links_H1.txt", "w+", encoding="utf8")

for i in range(len(countList)):
    resp = requests.get(countList[i][0])
    txt = resp.text
    soup = BeautifulSoup(txt, 'lxml')
    for i in range(len(soup.find_all('h1'))):
        if i != "JavaScript is not available.":
            f.write(soup.find_all('h1')[i].text)
            f.write("\n")

f.close()

In [36]:
import re
import pandas as pd
import feather
from collections import Counter
import requests
from bs4 import BeautifulSoup


# adjustable file in ftr format
save_dir = "allLinks.03.2020.EnglishTweets.ftr"

# reads contents of save_dir.ftr
read_ftr = pd.read_feather(save_dir, columns=None, use_threads=True);

# finds 100 most common words
countList = Counter(" ".join(read_ftr["urls"]).split()).most_common(100)
#print(countList)

previous = ""

for i in range(len(countList)):
    resp = requests.get(countList[i][0])
    txt = resp.text
    soup = BeautifulSoup(txt, 'lxml')
    finding = soup.body.find_all('img', alt=True)
    searchFor = 'alt'
    
    with open("english_Links_Img_Alt.txt", "a", encoding="utf8") as f:
        for j in finding:
            if j.attrs[searchFor] == previous:
                continue
            elif len(j.attrs[searchFor]) > 10:
                previous = j.attrs[searchFor]
                f.write(j.attrs[searchFor])
                f.write("\n")

In [ ]:
from nltk import *
import os

f = open('english_Links_H1.txt', 'r', encoding='utf-8')
raw = f.read()
tokens = word_tokenize(raw)
text = nltk.Text(tokens)